# Merge HyP3 ISCE2 burst InSAR products

This notebook demonstrates the `merge_tops_burst` workflow in the hyp3-isce2 plugin. The workflow merges multiple Burst InSAR Products. The input of the workflow is the directory that includes multiple HyP3-ISCE2 Burst InSAR Products.These products are created by HyP3-ISCE2 plugin. How to create HyP3 Burst InSAR Product is described in detail in the notebook [hyp3_isce2_burst_stack_for_ts_analysis](https://github.com/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_isce2_burst_stack_for_ts_analysis.ipynb).



**Note:** This notebook does assume you have some familiarity with InSAR processing with MintPy already, and is a minimal example without much context or explanations. If you're new to InSAR and MintPy, we suggest checking out the following resources. Note that some of these resources may be specific to our InSAR GAMMA products, so you may need to adapt them for use with our ISCE2-based burst InSAR products.

* Our [Burst Data Download Story Map](https://storymaps.arcgis.com/stories/88c8fe67933340779eddef212d76b8b8)

* Our [product guide](https://hyp3-docs.asf.alaska.edu/guides/burst_insar_product_guide/) for burst InSAR products

* Our [GitHub repository](https://github.com/asfhyp3/hyp3-isce2) containg the workflow used to create burst InSAR products

* Our [InSAR on Demand Story Map](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)

* [OpenSARlab's](https://opensarlab-docs.asf.alaska.edu/) highly detailed walkthrough of using HyP3 + MintPy via these notebooks:
  * [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
  * [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
  
    Note: While these notebooks make some assumptions you're working in OpenSARlab, you can run these 
    notebooks outside OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml).

## 0. Initial Setup
To run this notebook, you will need a local copy of the HyP3-ISCE2 GitHub repository and to set up a conda environment with the required dependencies. In your terminal, you can do this with the following commnads:

```shell
git clone https://github.com/ASFHyP3/hyp3-isce2.git
cd hyp3-isce2
mamba env create -f environment.yml
mamba activate hyp3-isce2
python -m pip install -e .
mamba install -c conda-forge pandas jupyter ipympl
jupyter notebook hyp3_isce2_burst_merge.ipynb
```

This workflow requires an Earth Data Cloud login. If you haven't yet, you can make an [account for free](https://urs.earthdata.nasa.gov/users/new) and set up a `.netrc` file in your home directory with your personal username and password.

```shell
echo ‘machine urs.earthdata.nasa.gov login $USERNAME password $PASSWORD’ >> ~/.netrc 
```

## 1. Prepare the input data
Before using the [HyP3-ISCE2 merge burst workflow](https://hyp3-docs.asf.alaska.edu/guides/burst_insar_product_guide/#merge-sentinel-1-burst-insar-products), we must double check our data are prepared so that all Bursts:
- Have the same reference and secondary dates
- Have the same polarization
- Have the same multilooking
- Be from the same relative orbit
- Be contiguous

We will set up a directory to store input data and output merge products.

In [ ]:
import os
from pathlib import Path

project_name = 'merge_burst_products'
work_dir = Path.cwd() / project_name
data_dir = work_dir / 'data'
if not data_dir.exists():
    data_dir.mkdir(parents=True, exist_ok=True)

Copy over unzipped folders that contain all outputs from HyP3 Burst processing to the data_dir directory. 

## 2. Merge the products using hyp3-isce2 merge_tops_burst workflow  

In [ ]:
import subprocess

cur_dir = Path.cwd()
os.chdir(work_dir)

cmd = ['python', '-m', 'hyp3_isce2', '++process', 'merge_tops_bursts', f'{str(data_dir)}', '--apply-water-mask', 'True']
subprocess.run(cmd)

os.chdir(cur_dir)

## 3. Display the merged product

In [ ]:
import rasterio
from rasterio.plot import show

tifs = [f for f in work_dir.glob("*/*.tif")]
for tif in tifs: 
    if not tif.name.split('_')[2]:
        src = rasterio.open(tif)
        show(src, title=tif.name)

In [ ]:
%matplotlib inline
import numpy as np
from osgeo import gdal
import matplotlib
import matplotlib.pyplot as plt

tifs = [f for f in work_dir.glob("*/*.tif")]

f, axs = plt.subplots(len(tifs), figsize=(6,6))
i = 0 
for tif in tifs: 
    if not tif.name.split('_')[2]:
        ds = gdal.Open(str(tif))
        merged_bursts = np.ma.masked_equal(ds.GetRasterBand(1).ReadAsArray(), 0)
        
    axs[i].imshow(merged_bursts)
    axs[i].set_title(tif.name)

    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
    plt.tight_layout()
    i = i + 1